In [1]:
import random
from random import shuffle
random.seed(1)

import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize.regexp import RegexpTokenizer
from nltk.tag import pos_tag

In [2]:
#stop word
stop_words = set(stopwords.words('english'))

#tokenizer
tokenizer = RegexpTokenizer("[\w']+")

In [3]:
import re
def get_only_chars(line):

    clean_line = ""

    line = line.replace("’", "")
    line = line.replace("'", "")
    line = line.replace("-", " ") #replace hyphens with spaces
    line = line.replace("\t", " ")
    line = line.replace("\n", " ")
    line = line.lower()

    for char in line:
        if char in 'qwertyuiopasdfghjklzxcvbnm':
            clean_line += char
        else:
            clean_line += ' '

    clean_line = re.sub(' +',' ',clean_line) #delete extra spaces
    if clean_line[0] == ' ':
        clean_line = clean_line[1:]
    return clean_line

In [4]:
#상위어, 유의어 대체
def synonym_replacement(words, n, sentences):
	new_words = words.copy()
	random_word_list = list(set([word for word in words if word not in stop_words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			#print("replaced", random_word, "with", synonym)
			num_replaced += 1
		if num_replaced >= n: #only replace up to n words
			break

	#this is stupid but we need it, trust me
	sentence = ' '.join(new_words)
	new_words = sentence.split(' ')

	return new_words


def get_synonyms(word):
	synonyms = set()
	tag = pos_tag(tokenizer.tokenize(sentences))

	noun_list = []
	etc_list = []

	for t in tag:
		if t[1] in ['NN', 'NNS', 'NNP', 'NNPS', 'PRP', 'PRP$']:
			noun_list.append(t[0])
		else:
			etc_list.append(t[0])

	if word in noun_list:
		i = noun_list.index(word)
		noun_word = noun_list[i]

		for syn in wordnet.synsets(noun_word):
			for l in syn.hypernyms(): 
				synonym = l.name().replace("_", " ").replace("-", " ").lower()
				synonym = "".join([char for char in synonym if char in 'qwertyuiopasdfghjklzxcvbnm'])
				synonym = synonym[:-1]
				synonyms.add(synonym)
	else:
		i = etc_list.index(word)
		etc_word = etc_list[i]
		for syn in wordnet.synsets(etc_word):
			for l in syn.lemmas(): 
				synonym = l.name().replace("_", " ").replace("-", " ").lower()
				#synonym = "".join([char for char in synonym if char in 'qwertyuiopasdfghjklzxcvbnm'])
				synonyms.add(synonym)
				
	if word in synonyms:
		synonyms.remove(word)
	return list(synonyms)


def eda(sentence, alpha_sr, num_aug):
    
    sentences = get_only_chars(sentence)
    words = tokenizer.tokenize(sentences)
    #words = [word for word in words if word is not '']
    num_words = len(words)
    
    augmented_sentences = []
    num_new_per_technique = int(num_aug/4)+1
    n_sr = max(1, int(alpha_sr*num_words))

    #sr
    for _ in range(num_new_per_technique):
        a_words = synonym_replacement(words, n_sr, sentences)
        augmented_sentences.append(' '.join(a_words))

    augmented_sentences = [get_only_chars(sentences) for sentences in augmented_sentences]
    shuffle(augmented_sentences)

    #trim so that we have the desired number of augmented sentences
    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

    #append the original sentence
    augmented_sentences.append(sentences)

    return augmented_sentences

In [ ]:
#유의어 대체
"""
def synonym_replacement(words, n, sentences):
	new_words = words.copy()
	random_word_list = list(set([word for word in words if word not in stop_words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			#print("replaced", random_word, "with", synonym)
			num_replaced += 1
		if num_replaced >= n: #only replace up to n words
			break

	#this is stupid but we need it, trust me
	sentence = ' '.join(new_words)
	new_words = sentence.split(' ')

	return new_words


def get_synonyms(word):
	synonyms = set()
	for syn in wordnet.synsets(word): 
		for l in syn.lemmas(): 
			synonym = l.name().replace("_", " ").replace("-", " ").lower()
			synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
			synonyms.add(synonym) 
	if word in synonyms:
		synonyms.remove(word)
	return list(synonyms)


def eda(sentence, alpha_sr, num_aug):
    
    sentences = get_only_chars(sentence)
    words = tokenizer.tokenize(sentences)
    #words = [word for word in words if word is not '']
    num_words = len(words)
    
    augmented_sentences = []
    num_new_per_technique = int(num_aug/4)+1
    n_sr = max(1, int(alpha_sr*num_words))

    #sr
    for _ in range(num_new_per_technique):
        a_words = synonym_replacement(words, n_sr, sentences)
        augmented_sentences.append(' '.join(a_words))

    augmented_sentences = [get_only_chars(sentences) for sentences in augmented_sentences]
    shuffle(augmented_sentences)

    #trim so that we have the desired number of augmented sentences
    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

    #append the original sentence
    augmented_sentences.append(sentences)

    return augmented_sentences
"""

In [14]:
sentences = 'a flat screen and a keyboard and mouse on the desk'
words = tokenizer.tokenize(sentences)
tag = pos_tag(words)
tag

[('a', 'DT'),
 ('flat', 'JJ'),
 ('screen', 'NN'),
 ('and', 'CC'),
 ('a', 'DT'),
 ('keyboard', 'NN'),
 ('and', 'CC'),
 ('mouse', 'NN'),
 ('on', 'IN'),
 ('the', 'DT'),
 ('desk', 'NN')]

In [16]:
synonym_replacement(words, 1, sentences)

['a',
 'flat',
 'screen',
 'and',
 'a',
 'keyboard',
 'and',
 'bruise',
 'on',
 'the',
 'desk']

In [19]:
new_sentences = 'a compressed screen and a keyboard and mouse on the desk'
new_words = tokenizer.tokenize(new_sentences)
new_tag = pos_tag(new_words)
new_tag

[('a', 'DT'),
 ('compressed', 'JJ'),
 ('screen', 'NN'),
 ('and', 'CC'),
 ('a', 'DT'),
 ('keyboard', 'NN'),
 ('and', 'CC'),
 ('mouse', 'NN'),
 ('on', 'IN'),
 ('the', 'DT'),
 ('desk', 'NN')]

In [ ]:
words

In [ ]:
new_words = words.copy()
random_word_list = list(set([word for word in words if word not in stop_words]))
random.shuffle(random_word_list)
random_word_list

In [17]:
synonyms = set()
for syn in wordnet.synsets('flat'):
    for l in syn.lemmas():
        synonym = l.name().replace("_", " ").replace("-", " ").lower()
        synonyms.add(synonym)
print(synonyms)

{'flat tire', 'directly', 'mat', 'prostrate', 'matt', 'flatcar', 'two dimensional', 'flatbed', 'matted', 'straight', 'level', 'plane', 'flavourless', 'flat', '2 dimensional', 'categoric', 'savorless', 'vapid', 'unconditional', 'flavorless', 'compressed', 'monotonic', 'bland', 'apartment', 'savourless', 'monotone', 'monotonous', 'insipid', 'categorical', 'matte'}


In [20]:
synonyms = set()
for syn in wordnet.synsets('screen'):
    for l in syn.hypernyms():
        synonym = l.name().replace("_", " ").replace("-", " ").lower()
        synonym = "".join([char for char in synonym if char in 'qwertyuiopasdfghjklzxcvbnm'])
        synonym = synonym[:-1]
        synonyms.add(synonym)
print(synonyms)

{'protectivecovering', 'door', 'strainer', 'covering', 'obstruct', 'sift', 'choose', 'display', 'analyze', 'check', 'show', 'partition', 'filmindustry', 'surface', 'protect'}


In [ ]:
for syn in wordnet.synsets('car'):
    for l in syn.hypernyms():
        print(l)

In [ ]:
for syn in wordnet.synsets('motor_vehicle'):
    for l in syn.lemmas():
        print(l)

In [ ]:
target_word = 'car.n.01'
print(wordnet.synset(target_word).hypernyms())
print()
for i, path in enumerate(wordnet.synset(target_word).hypernym_paths()):
    print(path)
    print()

In [ ]:
def find_hypernym_paths(target_word):
    syns = wordnet.synset(target_word)
    r_paths = []
    def hypernym_paths_helper(input_syns, paths):
        if len(input_syns.hypernyms())==0:
            for path in paths:
                r_paths.append(path)
        else:
            for hypernym in input_syns.hypernyms():
                hypernym_paths_helper(hypernym, [path+[hypernym] for path in paths])
    hypernym_paths_helper(syns, [[]])
    return r_paths
for i, path in enumerate(find_hypernym_paths('car.n.01')):
    print("path {}: {}".format(i, path))

In [ ]:
"""
from nltk.corpus import wordnet

word = 'man.n.01'

print(wordnet.synset(word).hypernyms())
print()
for i, path in enumerate(wordnet.synset(word).hypernym_paths()):
    print(path)
    print()
"""

In [ ]:
"""
word = 'sport_utility.n.01'

print(wordnet.synset(word).hypernyms())
print(wordnet.synset(word).lemmas())
"""

In [ ]:
#import nltk

In [ ]:
#nltk.download('wordnet')
#nltk.download('stopwords')
#nltk.download('averaged_perceptron_tagger')

In [ ]:
#arguments to be parsed from command line
"""
ap = argparse.ArgumentParser()
ap.add_argument("--input", required=True, type=str, help="input file of unaugmented data")
ap.add_argument("--output", required=False, type=str, help="output file of unaugmented data")
ap.add_argument("--num_aug", required=False, type=int, help="number of augmented sentences per original sentence")
ap.add_argument("--alpha", required=False, type=float, help="percent of words in each sentence to be changed")
args = ap.parse_args()
"""

In [ ]:
"""
from nltk.corpus import wordnet

word = 'phone'
pos = []
for syn in wordnet.synsets(word):
    pos.add(syn.pos())
    if pos[:1] == 'v':
        print('verb')
    elif pos[:1] == 'n':
        print('noun')
    else:
        print('?')
"""

<h2>Caption Augmentation_train data</h2>

In [ ]:
from load_caption import *
import easydict

In [ ]:
args = easydict.EasyDict({
    "input": 'D:/manuscript test/input/',
    "output": 'D:/manuscript test/output/',
    "num_aug": 2,
    "alpha": 0.1
})

In [ ]:
output = None
output = args.output
num_aug = args.num_aug
alpha = args.alpha

In [ ]:
"""
import pickle

with open('D:/coco text/train_filenames.pickle', 'rb') as f:
    train_filenames = pickle.load(f)
"""

In [ ]:
#generate more data with standard augmentation
def gen_eda(train_orig, output_file, alpha, num_aug=9):
    writer = open(output_file + 'word based.txt', 'w')
    lines = open(train_orig + 'word based.txt', 'r').readlines()

    for i, line in enumerate(lines):
        parts = line[:-1].split('\t')
        label = parts[0]
        sentence = parts[1]
        aug_sentences = eda(sentence, alpha, num_aug)
        for aug_sentence in aug_sentences:
            writer.write(aug_sentence + '\n')

    writer.close()

In [ ]:
#main function
if __name__ == "__main__":

    #generate augmented sentences and output into a new file
    gen_eda(args.input, output, alpha=alpha, num_aug=num_aug)